In [ ]:

import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm 
import wandb
from torch.optim.lr_scheduler import CosineAnnealingLR
import sys
from pathlib import Path
from torch.amp import autocast, GradScaler


sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..", "..")))
from src.utils.k_folds_creator import KFoldCreator
from src.utils.utils import get_models_save_path
from src.utils.constants import Constants 
from src.datasets.eeg_dataset_montage import EEGDatasetMontage
from src.models.gru_convolution_attention import NodeAttentionModel

wandb.login()


/home/maiko/miniconda3/envs/aicomp_312/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/home/maiko/miniconda3/envs/aicomp_312/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happ

Skipping module cbramod_dataset due to missing dependency: No module named 'mne'


wandb: Currently logged in as: maikotrede (hms-hslu-aicomp-hs25) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:

class CFG:
    seed = 42
    n_splits = 5
    data_path = '../../../data/'

    model_name = 'GRUConvNodeAttentionModel'
    hidden_units = 256
    num_layers = 1
    target_size = 6 
    
    num_cnn_blocks = 3 
    
    sampling_rate = 200 # Hz
    sequence_duration = 50 
    downsample_factor = 1
    
    num_channels = 19
    
    dropout = 0.4
    batch_size = 32
    num_workers = 8
    
    folds_to_train = [0,1,2,3,4] #speficy list of folds to train [0,1,2,3,4]
    

    stage1_epochs = 50
    stage1_lr = 10**-4
    
    stage2_epochs = 15
    stage2_lr = 10**-4.5
    
    patience = 10  
    min_delta = 0.001
    
    use_attention = True

    use_mixup = True      
    mixup_alpha = 0.5      
    

CFG.sequence_length = CFG.sequence_duration * CFG.sampling_rate 

def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)

set_seed(CFG.seed)

TARGETS = Constants.TARGETS



In [ ]:
def mixup_data(x, y, alpha=1.0, device='cuda'):
    """
    Applies MixUp to inputs and targets.
    Returns mixed inputs and mixed targets.
    """
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size(0)
    index = torch.randperm(batch_size).to(device)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    mixed_y = lam * y + (1 - lam) * y[index, :]
    
    return mixed_x, mixed_y


def get_dataloaders(train_df, valid_df):
    """
    Updated to handle empty DataFrames safely.
    Returns None for the loader if the input DataFrame is empty.
    """
    train_loader = None
    valid_loader = None


    if train_df is not None and not train_df.empty:
        train_dataset = EEGDatasetMontage(
            df=train_df, 
            data_path=CFG.data_path, 
            mode='train', 
            downsample_factor=CFG.downsample_factor, 
            augmentations=["channel_mask", "time_shift"] 
        )

        train_loader = DataLoader(
            train_dataset, batch_size=CFG.batch_size, shuffle=True,
            num_workers=CFG.num_workers, pin_memory=True, drop_last=True, 
            persistent_workers=True if CFG.num_workers > 0 else False
        )
    
    if valid_df is not None and not valid_df.empty:
        valid_dataset = EEGDatasetMontage(
            df=valid_df, 
            data_path=CFG.data_path, 
            mode='valid', 
            downsample_factor=CFG.downsample_factor
        )
        
        valid_loader = DataLoader(
            valid_dataset, batch_size=CFG.batch_size, shuffle=False,
            num_workers=CFG.num_workers, pin_memory=True, drop_last=False, 
            persistent_workers=True if CFG.num_workers > 0 else False
        )
    
    return train_loader, valid_loader




In [ ]:
def train_one_stage(fold, stage_name, train_df, valid_df, group_name, starting_checkpoint=None):

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    autocast_enabled = (device.type == 'cuda')
    
    if stage_name == "Stage1":
        lr = CFG.stage1_lr
        epochs = CFG.stage1_epochs
    else:
        lr = CFG.stage2_lr
        epochs = CFG.stage2_epochs
        
    print(f"\n--- Starting {stage_name} | Fold {fold} ---")
    

    experiment_name = f"{group_name}_{stage_name}_fold{fold}"
    
    config = {
        "architecture": CFG.model_name,
        "fold": fold, 
        "stage": stage_name,
        "optimizer": "AdamW", 
        "learning_rate": lr, 
        "batch_size": CFG.batch_size,
        "epochs": epochs, 
        "num_cnn_blocks": CFG.num_cnn_blocks,
        "use_attention": CFG.use_attention,
        "seed": CFG.seed
    }

    wandb.init(
        project="hms-aicomp-gru-conv",
        name=experiment_name,
        group=group_name, 
        job_type=stage_name,
        tags=['two-stage', stage_name, f'fold{fold}', f'blocks_{CFG.num_cnn_blocks}'],
        config=config,
        reinit=True
    )


    model = NodeAttentionModel(
        num_nodes=CFG.num_channels,       
        node_embed_size=256,              
        hidden_size=CFG.hidden_units,    
        num_layers=CFG.num_layers,       
        num_classes=CFG.target_size,     
        num_cnn_blocks=CFG.num_cnn_blocks,
        dropout=CFG.dropout,
        use_inception=True              
    )
    model.to(device)
    
    if starting_checkpoint:
        print(f"Loading weights from {starting_checkpoint}...")
        model.load_state_dict(torch.load(starting_checkpoint))

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-2)
    scheduler = CosineAnnealingLR(optimizer, T_max=epochs)
    loss_fn = nn.KLDivLoss(reduction='batchmean')
    scaler = GradScaler(enabled=autocast_enabled)

    train_loader, valid_loader = get_dataloaders(train_df, valid_df)

    best_val_loss = float('inf')
    best_model_path = get_models_save_path() / "TwoStage" / f"{stage_name}_fold{fold}.pth"
    best_model_path.parent.mkdir(parents=True, exist_ok=True)
    
    patience_counter = 0
    
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        
        for signals, labels in tqdm(train_loader, desc=f"{stage_name} E{epoch+1}", leave=False):
            signals, labels = signals.to(device), labels.to(device)
            
            if CFG.use_mixup:
                signals, labels = mixup_data(signals, labels, alpha=CFG.mixup_alpha, device=device)
            
            optimizer.zero_grad()
            with autocast(enabled=autocast_enabled, device_type=device.type):
                outputs = model(signals)
                log_probs = F.log_softmax(outputs, dim=1)
                loss = loss_fn(log_probs, labels)

            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            scaler.step(optimizer)
            scaler.update()

            train_loss += loss.item() * signals.size(0)
            
        train_loss /= len(train_loader.dataset)

        model.eval()
        valid_loss = 0
        with torch.no_grad():
            for signals, labels in valid_loader:
                signals, labels = signals.to(device), labels.to(device)
                with autocast(enabled=autocast_enabled, device_type=device.type):
                    outputs = model(signals)
                    log_probs = F.log_softmax(outputs, dim=1)
                    loss = loss_fn(log_probs, labels)
                valid_loss += loss.item() * signals.size(0)

        valid_loss /= len(valid_loader.dataset)

        epoch_lr = optimizer.param_groups[0]['lr']
        print(f"  Ep {epoch+1}: Train={train_loss:.4f} | Val={valid_loss:.4f} | LR={epoch_lr:.6f}")

        wandb.log({
            "epoch": epoch + 1, 
            "train/epoch_loss": train_loss,   
            "val/loss": valid_loss,          
            "val/kl_div": valid_loss,        
            "train/epoch_lr": epoch_lr       
        })

        if valid_loss < best_val_loss - CFG.min_delta:
            best_val_loss = valid_loss
            patience_counter = 0
            torch.save(model.state_dict(), best_model_path)
        else:
            patience_counter += 1
            
        if patience_counter >= CFG.patience:
            print(f"  Early stopping at epoch {epoch+1}")
            break 
            
        scheduler.step()
        
    wandb.finish()
    return best_model_path, best_val_loss



In [ ]:
def run_two_stage_pipeline(df):

    print("Total Votes Distribution (Head):")
    print(df['total_votes'].head())
    
    mask_low_votes = df['total_votes'] < 10
    mask_high_votes = df['total_votes'] >= 10
    
    print(f"Stage 1 Data (Low Votes < 10): {mask_low_votes.sum()} samples")
    print(f"Stage 2 Data (High Votes >= 10): {mask_high_votes.sum()} samples")
    group_name = f"TwoStage_montages_block_{CFG.num_cnn_blocks}_attention_{CFG.use_attention}_AUG(MU+CU)"
    
    all_oof_preds = []
    all_oof_labels = []
    fold_scores = []
    
    for fold in range(CFG.n_splits):
        print(f"\n{'='*20} Processing FOLD {fold} {'='*20}")
        if fold not in CFG.folds_to_train:
            print(f"Skipping Fold {fold}...")
            continue
        valid_idx = df['fold'] == fold
        valid_df = df[valid_idx].reset_index(drop=True)
        
        valid_stage2_df = valid_df[valid_df['total_votes'] >= 10].reset_index(drop=True)

        train_stage1 = df[(df['fold'] != fold) & mask_low_votes].reset_index(drop=True)
        
        stage1_path, _ = train_one_stage(
            fold=fold,
            stage_name="Stage1",
            train_df=train_stage1,
            valid_df=valid_df,
            group_name=group_name 
        )

        train_stage2 = df[(df['fold'] != fold) & mask_high_votes].reset_index(drop=True)
        
        stage2_path, best_val_loss = train_one_stage(
            fold=fold,
            stage_name="Stage2",
            train_df=train_stage2,
            valid_df=valid_stage2_df, 
            starting_checkpoint=stage1_path,
            group_name=group_name
        )
        
        fold_scores.append(best_val_loss)

        print(f"Generating OOF predictions for Fold {fold}...")
        device = torch.device('cuda')
        model = NodeAttentionModel(
            num_nodes=CFG.num_channels, node_embed_size=256, hidden_size=CFG.hidden_units,    
            num_layers=CFG.num_layers, num_classes=CFG.target_size, num_cnn_blocks=CFG.num_cnn_blocks,
            dropout=CFG.dropout, use_inception=True              
        )
        model.load_state_dict(torch.load(stage2_path))
        model.to(device)
        model.eval()
        
        _, valid_loader = get_dataloaders(pd.DataFrame(), valid_df)
        
        probs_list = []
        labels_list = []
        
        with torch.no_grad():
            for signals, labels in valid_loader:
                signals = signals.to(device)
                outputs = model(signals)
                probs = F.softmax(outputs, dim=1).cpu().numpy()
                probs_list.append(probs)
                labels_list.append(labels.numpy())
        
        all_oof_preds.append(np.concatenate(probs_list))
        all_oof_labels.append(np.concatenate(labels_list))
        
        del model
        torch.cuda.empty_cache()

        
        run = wandb.init(project="hms-aicomp-gru-conv", job_type="artifact_upload", name=f"artifact_fold{fold}")
        artifact = wandb.Artifact(f'model-fold{fold}-stage2', type='model')
        artifact.add_file(stage2_path)
        wandb.log_artifact(artifact)
        wandb.finish()

    all_oof_preds = np.concatenate(all_oof_preds)
    all_oof_labels = np.concatenate(all_oof_labels)
    
    oof_tensor = torch.tensor(all_oof_preds, dtype=torch.float32)
    true_tensor = torch.tensor(all_oof_labels, dtype=torch.float32)
    
    oof_tensor = torch.clamp(oof_tensor, 1e-6, 1.0)
    
    kl_loss = nn.KLDivLoss(reduction="batchmean")
    overall_score = kl_loss(torch.log(oof_tensor), true_tensor).item()
    
    return overall_score, fold_scores


In [ ]:

if __name__ == '__main__':
    print("Loading Data...")
    df = pd.read_csv(CFG.data_path + 'processed_data_max_vote_window.csv') 

    if 'expert_consensus' not in df.columns:
        df['expert_consensus'] = df[TARGETS].idxmax(axis=1)

    print('Train shape:', df.shape)

    fold_creator = KFoldCreator(n_splits=CFG.n_splits, seed=CFG.seed)
    df = fold_creator.create_folds(df, stratify_col='expert_consensus', group_col='patient_id')

    overall_cv, fold_scores = run_two_stage_pipeline(df)
    
    print("\n" + "="*50)
    print("FINAL RESULTS")
    print("="*50)
    print(f"Overall OOF KL-Divergence: {overall_cv:.4f}")
    print(f"Average Fold Score: {np.mean(fold_scores):.4f}")
    print("="*50)